<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/Image/14_registering_images.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/Image/14_registering_images.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/Image/14_registering_images.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Registering Images
The Earth Engine image registration algorithm is designed to be a final, post-ortho, fine-grained step in aligning images. It is assumed that the images to be registered have already gone through initial alignment stages, so they are already within a few degrees of rotation of one another, and differ by only small translations. The registration uses a “rubber-sheet” technique, allowing local image warping to correct for orthorectification errors and other artifacts from earlier processing. The underlying alignment technique is image correlation, so the bands for the input and reference images must be visually similar in order for the algorithm to compute an accurate alignment.

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

## Image displacement
There are two steps to registering an image: Determining the displacement image using `displacement()`, and then applying it with `displace()`. The required inputs are the pair of images to register, and a maximum displacement parameter (`maxOffset`).

The `displacement()` algorithm takes a reference image, a maximum displacement parameter (`maxOffset`), and two optional parameters that modify the algorithm behaviour. The output is a displacement image with bands `dx` and `dy` which give the X and Y components (in meters) of the displacement vector at each pixel.

All bands of the calling and reference images are used for matching during registration, so the number of bands must be exactly equal. The input bands must be visually similar for registration to succeed. If that is not the case, it may be possible to pre-process them (e.g. smoothing, edge detection) to make them appear more similar. The registration computations are performed using a multiscale, coarse-to-fine process, with (multiscale) working projections that depend on three of the projections supplied to the algorithm:

1. the default projection of the calling image (Pc)
2. the default projection of the reference image (Pr)
3. the output projection (Po)

The highest resolution working projection (*Pw* will be in the CRS of *Pr*, at a scale determined by the coarsest resolution of these 3 projections, to minimize computation. The results from *Pr* are then resampled to be in the projection specified by the input ‘projection’ parameter.

The output is a displacement image with the following bands:

`dx`

* For a given reference image pixel location, this band contains the distance in the X direction that must be travelled to arrive at the matching location in the calling image. Units are in geodesic meters.

`dy`
* For a given reference image pixel location, this band contains the distance in the Y direction that must be travelled to arrive at the matching location in the calling image. Units are in geodesic meters.

`confidence`

* This is a per-pixel estimate of displacement confidence (where 0 is low confidence and 1 is high confidence) based on the correlation scores in regions where valid matches were found. In regions where no matches were found, confidence is estimated from nearby correlations using a Gaussian kernel to provide higher weight to nearby correlations.
The following example computes the magnitude and angle of displacement between two high-resolution [Terra Bella](https://terrabella.google.com/) images:

In [ ]:
import math

# Load the two images to be registered.
image1 = ee.Image("SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150502T082736Z")
image2 = ee.Image("SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150305T081019Z")

# Use bicubic resampling during registration.
image1Orig = image1.resample("bicubic")
image2Orig = image2.resample("bicubic")

# Choose to register using only the 'R' bAnd.
image1RedBAnd = image1Orig.select("R")
image2RedBAnd = image2Orig.select("R")

# Determine the displacement by matching only the 'R' bAnds.
displacement = image2RedBAnd.displacement(
    **{"referenceImage": image1RedBAnd, "maxOffset": 50.0, "patchWidth": 100.0}
)

# Compute image offset And direction.
offset = displacement.select("dx").hypot(displacement.select("dy"))
angle = displacement.select("dx").atan2(displacement.select("dy"))

# Display offset distance And angle.
Map.addLayer(offset, {"min": 0, "max": 20}, "offset")
Map.addLayer(angle, {"min": -math.pi, "max": math.pi}, "angle")
Map.setCenter(37.44, 0.58, 15)

## Warping an image
There are two ways to warp an image to match another image: `displace()` or `register()`. The `displace()` algorithm takes a displacement image having `dx` and `dy` bands as the first two bands, and warps the image accordingly. The output image will be the result of warping the bands of the input image by the offsets present in the displacement image. Using the displacements computed in the previous example:

In [ ]:
# Use the computed displacement to register all Original bAnds.
registered = image2Orig.displace(displacement)

# Show the results of co-registering the images.
visParams = {"bands": ["R", "G", "B"], "max": 4000}
Map.addLayer(image1Orig, visParams, "Reference")
Map.addLayer(image2Orig, visParams, "BefOre Registration")
Map.addLayer(registered, visParams, "After Registration")

If you don't need the displacement bands, Earth Engine provides the `register()` method, which is a shortcut for calling `displacement()` followed by `displace()`. For example:

In [ ]:
alsoRegistered = image2Orig.register(
    **{"referenceImage": image1Orig, "maxOffset": 50.0, "patchWidth": 100.0}
)
Map.addLayer(alsoRegistered, visParams, "Also Registered")

In this example, the results of `register()` differ from the results of `displace()`. This is because a different set of bands was used in the two approaches: `register()` always uses all bands of the input images, while the `displacement()` example used only the red band before feeding the result to `displace()`. Note that when multiple bands are used, if band variances are very different this could over-weight the high-variance bands, since the bands are jointly normalized when their spatial correlation scores are combined. This illustrates the importance of selecting band(s) that are visually the most similar when registering. As in the previous example, use `displacement()` and `displace()` for control over which bands are used to compute displacement.

## Display Earth Engine data layers 

In [ ]:
Map.addLayerControl()
Map